In [1]:
import os
import xml.etree.ElementTree as ET
import re
from datetime import datetime,timedelta
import pytz
import csv

In [22]:
path = "data_info_main.csv"
with open(path,'w') as f:
    csv_write = csv.writer(f)
    csv_head = ["Time","Temperature","Relative Humidity"]
    # csv_head = np.array(csv_head)
    # str.encode(csv_head,'utf-8')
    csv_write.writerow(csv_head)

In [35]:
path = "data_info_area.csv"
with open(path,'w') as f:
    csv_write = csv.writer(f)
    csv_head = ["Time","Hong Kong Observatory","King's Park","Wong Chuk Hang","Ta Kwu Ling","Lau Fau Shan","Tai Po","Sha Tin","Tuen Mun","Tseung Kwan","Sai Kung","Cheung Chau","Chek Lap Kok","Tsing Yi","Shek Kong","Tsuen Wan Ho Koon","Tsuen Wan Shing Mun Valley","Hong Kong Park","Shau Kei Wan","Kowloon City","Happy Valley","Wong Tai Sin","Stanley","Kwun Tong","Sham Shui","Kai Tak Runway Park","Yuen Long Park","Tai Mei Tuk"]
    csv_write.writerow(csv_head)

In [54]:
path = "data_info_rain.csv"
with open(path,'w') as f:
    csv_write = csv.writer(f)
    csv_head = ["Time","area","rain"]
    csv_write.writerow(csv_head)

In [39]:
path = "data_info_reminder.csv"
with open(path,'w') as f:
    csv_write = csv.writer(f)
    csv_head = ["Time","reminder"]
    csv_write.writerow(csv_head)

In [42]:
path = "data_info_uv.csv"
with open(path,'w') as f:
    csv_write = csv.writer(f)
    csv_head = ["Time","UV","Intensity"]
    csv_write.writerow(csv_head)

In [5]:
def convert_to_beijing_time(gmt_time_str):
    
    gmt_timezone = pytz.timezone('GMT')

    gmt_time = datetime.strptime(gmt_time_str, '%a, %d %b %Y %H:%M:%S %Z').replace(tzinfo=gmt_timezone)
    
    beijing_timezone = pytz.timezone('Asia/Shanghai')
    
    beijing_time = gmt_time.astimezone(beijing_timezone)

    return beijing_time.strftime('%Y-%m-%d %H:%M:%S')

In [9]:
def write_xml_main(file_path, xml_path, file_name):

    tree = ET.parse(file_path)
    root = tree.getroot()
    
    data_row = []
    
    # Open the new file in write mode
    with open(file_name + '.txt', 'w') as f:
        for item in root.iter('item'):
            # Extract and write the time
            time = item.find('pubDate').text
            time = convert_to_beijing_time(time)
            f.write(f'Time: {time}\n')
            data_row.append(time)
    
            # Extract and write the air temperature and relative humidity
            description = item.find('description').text
            air_temp_match = re.search('Air temperature : (.*) degrees Celsius', description)
            rel_humidity_match = re.search('Relative Humidity : (.*) per cent', description)
            uv_match = re.search('the mean UV Index recorded at King\'s Park : (.*)<br/>', description)
            remind_match = re.search('reminded that:<br/>(.*)<br/></SPAN><br/>', description)
            air_temp = air_temp_match.group(1) if air_temp_match else 'NONE'
            rel_humidity = rel_humidity_match.group(1) if rel_humidity_match else 'NONE'
            uv = uv_match.group(1) if uv_match else 'NONE'
            remind = remind_match.group(1) if remind_match else 'NONE'
            f.write(f'Air Temperature: {air_temp} degrees Celsius\n')
            f.write(f'Relative Humidity: {rel_humidity} per cent\n')
            f.write(f'the mean UV Index recorded at King\'s Park : {uv}\n')
            f.write(f'remind that: {remind}\n')
            data_row.append(air_temp)
            data_row.append(rel_humidity)
            print(data_row)
    
            # Extract and write the air temperatures at other places
            f.write('Air Temperatures at Other Places:\n')
            for match in re.finditer('<tr><td><font size="-1">(.*?)</font></td><td width="100" align="right"><font size="-1">(.*?) ;</font></td></tr>', description):
                place_temp = match.group(1) + ' ' + match.group(2)
                f.write(f'{place_temp}\n')
    
            # Extract and write the rainfall recorded in various regions
            f.write('Rainfall Recorded in Various Regions:\n')
            for match in re.finditer('<tr><td>(.*?)</td><td width="100" align="right">(.*?)&nbsp;mm.', description):
                rainfall = match.group(1) + ' ' + match.group(2) + ' mm' 
                f.write(f'{rainfall}\n')
    
    # write the xml file
    with open(xml_path, 'a+') as f:
        csv_write = csv.writer(f)
        csv_write.writerow(data_row)

In [36]:
def write_xml_area(file_path, xml_path):

    tree = ET.parse(file_path)
    root = tree.getroot()
    
    data_row = []
    
    # Open the new file in write mode
    for item in root.iter('item'):
        time = item.find('pubDate').text
        time = convert_to_beijing_time(time)
        data_row.append(time)
    
        description = item.find('description').text
        
        # Extract and write the air temperatures at other places
        for match in re.finditer('<tr><td><font size="-1">(.*?)</font></td><td width="100" align="right"><font size="-1">(.*?) degrees ', description):
            place_temp = match.group(2)
            data_row.append(place_temp)
        print(data_row)
            
    # write the xml file
    if data_row.__len__() == 28:
        with open(xml_path, 'a+') as f:
            csv_write = csv.writer(f)
            csv_write.writerow(data_row)

In [55]:
def write_xml_rain(file_path, xml_path):

    tree = ET.parse(file_path)
    root = tree.getroot()
    
    
    # Open the new file in write mode
    for item in root.iter('item'):
        time = item.find('pubDate').text
        time = convert_to_beijing_time(time)
        
    
        description = item.find('description').text
        
            
        for match in re.finditer('<tr><td>(.*?)</td><td width="100" align="right">(.*?)&nbsp;mm.', description):
            data_row = []
            data_row.append(time)
            rain_place = match.group(1)
            rainfall = match.group(2)
            data_row.append(rain_place)
            data_row.append(rainfall)
            print(data_row)
            if data_row.__len__() != 1:
                with open(xml_path, 'a+') as f:
                    csv_write = csv.writer(f)
                    csv_write.writerow(data_row)
    

In [41]:
def write_xml_uv(file_path, xml_path):

    tree = ET.parse(file_path)
    root = tree.getroot()
    
    data_row = []
    
    # Open the new file in write mode
    for item in root.iter('item'):
        time = item.find('pubDate').text
        time = convert_to_beijing_time(time)
        data_row.append(time)
    
        description = item.find('description').text
        
        # Extract and write the air temperatures at other places
        uv_match = re.search('the mean UV Index recorded at King\'s Park : (.*)<br/>\n', description)
        uv = uv_match.group(1) if uv_match else 'NONE'
        if uv != 'NONE':
            data_row.append(uv)
            Intensity = re.search('Intensity of UV radiation : (.*)<br/>', description).group(1)
            if Intensity.__len__() > 20:
                Intensity = Intensity.split('<br/>',1)[0]
                data_row.append(Intensity)
            else:
                data_row.append(Intensity)
            print(data_row)
            
    # write the xml file
    if data_row.__len__() != 1:
        with open(xml_path, 'a+') as f:
            csv_write = csv.writer(f)
            csv_write.writerow(data_row)

In [6]:
path = '/Users/lyu/School/SemeterA/Big_Data/final_project/data'

path_list = os.listdir(path)

path_list.remove('.DS_Store')

path_list.sort()

print(path_list)

['20221001-0001.xml', '20221001-0101.xml', '20221001-0201.xml', '20221001-0301.xml', '20221001-0401.xml', '20221001-0501.xml', '20221001-0601.xml', '20221001-0701.xml', '20221001-0801.xml', '20221001-0901.xml', '20221001-1001.xml', '20221001-1101.xml', '20221001-1201.xml', '20221001-1301.xml', '20221001-1401.xml', '20221001-1501.xml', '20221001-1601.xml', '20221001-1701.xml', '20221001-1801.xml', '20221001-1901.xml', '20221001-2001.xml', '20221001-2101.xml', '20221001-2201.xml', '20221001-2301.xml', '20221002-0001.xml', '20221002-0101.xml', '20221002-0201.xml', '20221002-0301.xml', '20221002-0401.xml', '20221002-0501.xml', '20221002-0601.xml', '20221002-0701.xml', '20221002-0801.xml', '20221002-0901.xml', '20221002-1001.xml', '20221002-1101.xml', '20221002-1201.xml', '20221002-1301.xml', '20221002-1401.xml', '20221002-1501.xml', '20221002-1601.xml', '20221002-1701.xml', '20221002-1801.xml', '20221002-1901.xml', '20221002-2001.xml', '20221002-2101.xml', '20221002-2201.xml', '20221002-23

In [10]:
xml_path = '/Users/lyu/School/SemeterA/Big_Data/final_project/data_info_main.csv'

for f in path_list:
    file_path = '/Users/lyu/School/SemeterA/Big_Data/final_project/data/' + f
    file_name = f[0:13]
    final_file_name = '/Users/lyu/School/SemeterA/Big_Data/final_project/data_txt/' + file_name
    write_xml_main(file_path, xml_path, final_file_name)

In [37]:
xml_path = '/Users/lyu/School/SemeterA/Big_Data/final_project/data_info_area.csv'

for f in path_list:
    file_path = '/Users/lyu/School/SemeterA/Big_Data/final_project/data/' + f
    file_name = f[0:13]
    write_xml_area(file_path, xml_path)

['2022-09-30 23:11:00', '26', '25', '25', '24', '24', '24', '25', '25', '25', '25', '24', '26', '25', '24', '23', '24', '25', '25', '25', '25', '26', '26', '25', '25', '26', '24', '25']
['2022-10-01 00:02:00', '26', '25', '25', '25', '24', '25', '25', '24', '25', '25', '24', '26', '25', '25', '23', '24', '25', '25', '25', '25', '26', '26', '25', '25', '26', '24', '25']
['2022-10-01 01:02:00', '26', '25', '25', '25', '24', '25', '26', '25', '25', '25', '24', '26', '25', '26', '25', '24', '25', '25', '25', '25', '26', '25', '25', '26', '24', '25']
['2022-10-01 02:02:00', '26', '25', '26', '25', '24', '25', '26', '25', '25', '26', '24', '27', '25', '26', '24', '25', '24', '25', '25', '25', '26', '27', '25', '25', '26', '24', '25']
['2022-10-01 03:02:00', '26', '25', '26', '25', '24', '25', '26', '25', '25', '26', '25', '27', '25', '25', '24', '24', '25', '26', '25', '25', '27', '27', '26', '25', '26', '24', '25']
['2022-10-01 04:02:00', '26', '25', '26', '25', '24', '25', '26', '25', '25'

In [56]:
xml_path = '/Users/lyu/School/SemeterA/Big_Data/final_project/data_info_rain.csv'

for f in path_list:
    file_path = '/Users/lyu/School/SemeterA/Big_Data/final_project/data/' + f
    file_name = f[0:13]
    write_xml_rain(file_path, xml_path)

['2022-09-30 23:11:00', 'Islands District', '0 to 2']
['2022-10-01 00:02:00', 'Islands District', '0 to 2']
['2022-10-01 03:02:00', 'Tai Po', '0 to 1']
['2022-10-01 04:02:00', 'North District', '0 to 1']
['2022-10-01 09:02:00', 'Tai Po', '0 to 3']
['2022-10-01 09:02:00', 'Tsuen Wan', '0 to 1']
['2022-10-01 09:02:00', 'Wong Tai Sin', '0 to 1']
['2022-10-01 10:02:00', 'North District', '0 to 4']
['2022-10-01 10:02:00', 'Sha Tin', '0 to 3']
['2022-10-01 10:02:00', 'Tsuen Wan', '0 to 3']
['2022-10-01 10:02:00', 'Tai Po', '0 to 2']
['2022-10-01 10:02:00', 'Wan Chai', '0 to 2']
['2022-10-01 10:02:00', 'Wong Tai Sin', '0 to 2']
['2022-10-01 10:02:00', 'Yuen Long', '0 to 2']
['2022-10-01 10:02:00', 'Central &amp; Western District', '0 to 1']
['2022-10-01 11:02:00', 'Tai Po', '0 to 1']
['2022-10-01 12:02:00', 'Tai Po', '0 to 1']
['2022-10-01 13:30:00', 'Wan Chai', '1 to 5']
['2022-10-01 13:30:00', 'Eastern District', '1 to 3']
['2022-10-01 13:30:00', 'Central &amp; Western District', '0 to 2']


In [43]:
xml_path = '/Users/lyu/School/SemeterA/Big_Data/final_project/data_info_uv.csv'

for f in path_list:
    file_path = '/Users/lyu/School/SemeterA/Big_Data/final_project/data/' + f
    file_name = f[0:13]
    write_xml_uv(file_path, xml_path)

['2022-10-01 08:02:00', '0.3', 'low']
['2022-10-01 09:02:00', '2', 'low']
['2022-10-01 10:02:00', '4', 'moderate']
['2022-10-01 11:02:00', '5', 'moderate']
['2022-10-01 12:02:00', '6', 'high']
['2022-10-01 13:30:00', '6', 'high']
['2022-10-01 14:31:00', '6', 'high']
['2022-10-01 15:26:00', '3', 'moderate']
['2022-10-01 16:02:00', '1', 'low']
['2022-10-01 17:11:00', '0.6', 'low']
['2022-10-01 18:02:00', '0.1', 'low']
['2022-10-02 08:02:00', '0.4', 'low']
['2022-10-02 09:02:00', '1', 'low']
['2022-10-02 10:02:00', '3', 'moderate']
['2022-10-02 11:02:00', '5', 'moderate']
['2022-10-02 12:02:00', '6', 'high']
['2022-10-02 13:02:00', '5', 'moderate']
['2022-10-02 14:02:00', '7', 'high']
['2022-10-02 15:02:00', '5', 'moderate']
['2022-10-02 16:02:00', '3', 'moderate']
['2022-10-02 17:02:00', '0.9', 'low']
['2022-10-02 18:16:00', '0.2', 'low']
['2022-10-03 08:02:00', '0.5', 'low']
['2022-10-03 09:02:00', '2', 'low']
['2022-10-03 10:16:00', '4', 'moderate']
['2022-10-03 11:02:00', '7', 'high']